In [1]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from IPython.display import display, HTML
import pandas as pd
import numpy as np

# import observations of selected objects outputted midway from sorcha
obs_df = pd.read_csv('./observations.csv')
obs_df = obs_df.sort_values(by=['ssObjectId'])
obs_df = obs_df.reset_index(drop=True)
obs_df['ssObjectId'] = obs_df['ssObjectId'].astype(str)


# convert to record array for sorcha compatability
nameLen = obs_df['ssObjectId'].str.len().max()
obsv = obs_df.to_records(
    index=False,
    column_dtypes=dict(ssObjectId=f'a{nameLen}', diaSourceId='u8', midPointTai='f8', ra='f8', decl='f8')
)

# split record array by object
i = np.argsort(obsv['ssObjectId'], kind='stable')
ssObjects, idx = np.unique(obsv['ssObjectId'][i], return_index=True)
objSplits = np.split(i, idx[1:])

tracklets = []
a=0
no = 0

# select one object at a time to see if it gets linked
for a in range(len(ssObjects)):
    testdf = obs_df.loc[objSplits[a],:]
    testdf = testdf.sort_values(by=['midPointTai'])
    # display(HTML(testdf.to_html()))

    # split by each unique mjd date of observation and put into nights
    splits = np.split(testdf['midPointTai'].values, np.where(np.abs(np.diff(testdf['midPointTai'].values)) >= 0.5)[0] + 1)
    splitra = np.split(testdf['ra'].values, np.where(np.abs(np.diff(testdf['midPointTai'].values)) >= 0.5)[0] + 1)
    splitdec = np.split(testdf['decl'].values, np.where(np.abs(np.diff(testdf['midPointTai'].values)) >= 0.5)[0] + 1)
    ras = [list(x) for x in splitra if len(x) > 1]
    decs = [list(x) for x in splitdec if len(x) > 1]
    nights = [list(x) for x in splits if len(x) > 1]
    obspernight = [len(ns) for ns in nights]


    # loop over each night and test for tracklets
    for n, night in enumerate(nights):
        if obspernight[n] < 2:
            pass
        else:
            for i in np.arange(len(night)):
                for j in np.arange(len(night)):
                    if j <= i:
                        pass
                    else:
                        diff = night[j] - night[i]
                        if diff > 0 and diff < 90/(60*24):
                            coord1 = SkyCoord([ras[n][i]], [decs[n][i]], unit=u.deg) 
                            coord2 = SkyCoord([ras[n][j]], [decs[n][j]], unit=u.deg)
                            sep = coord1.separation(coord2).arcsecond
                            if sep > 0.5:
                                tracklet = dict(
                                    no=no,
                                    ssObjectId=ssObjects[a],
                                    ra1=ras[n][i],
                                    dec1=decs[n][i],
                                    ra2=ras[n][j],
                                    dec2=decs[n][j],
                                    mjd1=night[i],
                                    mjd2=night[j]
                                )
                                tracklets.append(tracklet)
                                no += 1

linkdf = pd.DataFrame(tracklets)

nameLen = linkdf['ssObjectId'].str.len().max()
links = linkdf.to_records(
    index=False,
    column_dtypes=dict(no='f8', ssObjectId=f'a{nameLen}', ra1='f8', dec1='f8', ra2='f8', dec2='f8', mjd1='f8', mjd2='f8')
)

# split record array by object
i = np.argsort(links['ssObjectId'], kind='stable')
ssObjects, idx = np.unique(links['ssObjectId'][i], return_index=True)
objSplits = np.split(i, idx[1:])

# manually check which tracklet pairs are within 15 days of each other
for a in range(len(ssObjects)):
    testdf = linkdf.loc[objSplits[a],:]
    testdf = testdf.sort_values(by=['mjd1'])
    # testdf.to_csv(f'/Users/josephmurtagh/Desktop/{ssObjects[a]}.csv', index=False)
    display(HTML(testdf.to_html()))
    for n, i in enumerate(np.arange(len(testdf))):
        for m, j in enumerate(np.arange(len(testdf))):
            if j <= i: # ensure we're not linking self with self or self with temporally past tracklets
                pass
            else:
                diff = testdf.iloc[j,:]['mjd2'] - testdf.iloc[i,:]['mjd1']
                if diff > 15: # if the difference is above 15 days, stop the checks and use mjds from before
                    idxs = np.arange(n, m)
                    if idxs.size != 0:
                        floors = [np.floor(x) for x in testdf.iloc[idxs,:]['mjd1']] # <-- makes sure that the tracklets are defined on seperate nights to avoid linking 3 tracklets on e.g. nights 1, 5.2, and 5.5
                        if len(np.unique(floors)) >= 3:
                            print('link between tracklet', testdf.iloc[i,:]['no'], testdf.iloc[i,:]['mjd1'], 'to tracklet', testdf.iloc[j-1,:]['no'], testdf.iloc[j-1,:]['mjd2'])
                    break


,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
0,0,b'1221',321.115533,-0.667670,321.107305,-0.673209,60524.295584,60524.319908
1,1,b'1221',320.480566,-1.116599,320.472296,-1.122284,60526.243081,60526.267457
2,2,b'1221',319.530763,-1.809127,319.530615,-1.809229,60529.171286,60529.171734
3,3,b'1221',319.530763,-1.809127,319.522551,-1.814984,60529.171286,60529.195743
4,4,b'1221',319.530615,-1.809229,319.522551,-1.814984,60529.171734,60529.195743
5,5,b'1221',314.636387,-5.923065,314.630455,-5.928636,60546.214165,60546.238237
6,6,b'1221',311.674221,-9.740484,311.672184,-9.744554,60565.009099,60565.033083
7,7,b'1221',311.674221,-9.740484,311.669799,-9.749304,60565.009099,60565.061132
8,8,b'1221',311.674221,-9.740484,311.669759,-9.749380,60565.009099,60565.061583
9,9,b'1221',311.672184,-9.744554,311.669799,-9.749304,60565.033083,60565.061132


link between tracklet 0 60524.295583660896 to tracklet 4 60529.195743488555


,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
12,12,b'2013SJ112',16.778282,2.957682,16.777814,2.957458,60229.286357,60229.310248
13,13,b'2013SJ112',16.672241,2.915144,16.671780,2.914928,60235.143978,60235.167713
14,14,b'2013SJ112',16.154161,2.724629,16.153828,2.724472,60267.055746,60267.080889
15,15,b'2013SJ112',16.154161,2.724629,16.153420,2.724409,60267.055746,60267.104683
16,16,b'2013SJ112',16.153828,2.724472,16.153420,2.724409,60267.080889,60267.104683
17,17,b'2013SJ112',15.912479,2.657293,15.912340,2.657280,60291.060287,60291.084191
18,18,b'2013SJ112',18.672378,3.775768,18.672055,3.775592,60549.305147,60549.329375
19,19,b'2013SJ112',18.441729,3.666732,18.440989,3.666413,60566.235666,60566.283615


,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
20,20,b'2014QN441',51.529939,-3.066424,51.529434,-3.066683,60236.203171,60236.227402
21,21,b'2014QN441',51.529939,-3.066424,51.529434,-3.066688,60236.203171,60236.227849
22,22,b'2014QN441',51.424972,-3.115546,51.424842,-3.115603,60241.240673,60241.246638
23,23,b'2014QN441',51.424972,-3.115546,51.424294,-3.115835,60241.240673,60241.271287
24,24,b'2014QN441',51.424842,-3.115603,51.424294,-3.115835,60241.246638,60241.271287
25,25,b'2014QN441',51.249766,-3.186002,51.249204,-3.186217,60249.162204,60249.186618
26,26,b'2014QN441',51.249766,-3.186002,51.248761,-3.186382,60249.162204,60249.205747
27,27,b'2014QN441',51.249768,-3.186009,51.249204,-3.186217,60249.162650,60249.186618
28,28,b'2014QN441',51.249768,-3.186009,51.248761,-3.186382,60249.162650,60249.205747
29,29,b'2014QN441',51.249204,-3.186217,51.248761,-3.186382,60249.186618,60249.205747


link between tracklet 20 60236.203170757246 to tracklet 30 60250.25729704931
link between tracklet 21 60236.203170757246 to tracklet 30 60250.25729704931
link between tracklet 22 60241.2406725659 to tracklet 30 60250.25729704931
link between tracklet 23 60241.2406725659 to tracklet 30 60250.25729704931
link between tracklet 24 60241.24663784267 to tracklet 30 60250.25729704931
link between tracklet 25 60249.16220389849 to tracklet 32 60261.24630923874
link between tracklet 26 60249.16220389849 to tracklet 32 60261.24630923874
link between tracklet 27 60249.162649768885 to tracklet 32 60261.24630923874
link between tracklet 28 60249.162649768885 to tracklet 32 60261.24630923874
link between tracklet 29 60249.186617577536 to tracklet 32 60261.24630923874
link between tracklet 30 60250.246012177086 to tracklet 33 60264.25241530691
link between tracklet 31 60260.2247868456 to tracklet 33 60264.25241530691


...

,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
54,54,b'90377',69.885704,10.429873,69.885453,10.429814,60261.269575,60261.293720
55,55,b'90377',69.824769,10.417903,69.824512,10.417852,60267.215889,60267.239587
56,56,b'90377',69.814261,10.415996,69.814010,10.415952,60268.218714,60268.242663
57,57,b'90377',69.814261,10.415996,69.813771,10.415912,60268.218714,60268.264818
58,58,b'90377',69.814010,10.415952,69.813771,10.415912,60268.242663,60268.264818
59,59,b'90377',69.686253,10.396315,69.685991,10.396282,60280.153376,60280.177049
60,60,b'90377',69.653186,10.392234,69.652913,10.392207,60283.194342,60283.218107
61,61,b'90377',69.653186,10.392234,69.652918,10.392204,60283.194342,60283.218553
62,62,b'90377',69.653179,10.392236,69.652913,10.392207,60283.194789,60283.218107
63,63,b'90377',69.653179,10.392236,69.652918,10.392204,60283.194789,60283.218553


link between tracklet 54 60261.26957518382 to tracklet 58 60268.26481755462
link between tracklet 55 60267.21588879789 to tracklet 59 60280.17704897061
link between tracklet 56 60268.21871410435 to tracklet 63 60283.21855319268
link between tracklet 57 60268.21871410435 to tracklet 63 60283.21855319268
link between tracklet 58 60268.24266321104 to tracklet 63 60283.21855319268


,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
68,68,b'c0',300.705298,4.688156,300.705208,4.687860,60224.015240,60224.039023
69,69,b'c0',300.705298,4.688156,300.705199,4.687833,60224.015240,60224.040362
70,70,b'c0',300.705290,4.688138,300.705208,4.687860,60224.016583,60224.039023
71,71,b'c0',300.705290,4.688138,300.705199,4.687833,60224.016583,60224.040362
72,72,b'c0',300.705278,4.688127,300.705208,4.687860,60224.017030,60224.039023
73,73,b'c0',300.705278,4.688127,300.705199,4.687833,60224.017030,60224.040362
74,74,b'c0',300.700146,4.662632,300.700123,4.662493,60225.997603,60226.008918
75,75,b'c0',303.509265,5.386203,303.508716,5.386133,60503.211770,60503.235748
76,76,b'c0',303.443319,5.375031,303.442842,5.374874,60506.129949,60506.153694
77,77,b'c0',303.009116,5.259785,303.008679,5.259642,60525.111749,60525.130951


,no,ssObjectId,ra1,dec1,ra2,dec2,mjd1,mjd2
86,86,b'c1',246.139338,9.081515,246.139343,9.081673,60383.389322,60383.399166
87,87,b'c1',245.860740,9.539122,245.860347,9.539535,60410.310708,60410.334502
88,88,b'c1',245.860740,9.539122,245.860089,9.539719,60410.310708,60410.348448
89,89,b'c1',245.860740,9.539122,245.859781,9.540023,60410.310708,60410.368128
90,90,b'c1',245.860347,9.539535,245.860089,9.539719,60410.334502,60410.348448
91,91,b'c1',245.860347,9.539535,245.859781,9.540023,60410.334502,60410.368128
92,92,b'c1',245.860089,9.539719,245.859781,9.540023,60410.348448,60410.368128
93,93,b'c1',245.826258,9.570483,245.825806,9.570843,60412.309195,60412.333408
95,95,b'c1',245.401664,9.845719,245.401560,9.845823,60432.242641,60432.247989
94,94,b'c1',245.401664,9.845719,245.401504,9.845723,60432.242641,60432.247542


link between tracklet 95 60432.242641181445 to tracklet 101 60443.30880202504
link between tracklet 94 60432.242641181445 to tracklet 101 60443.30880202504
